**Ensemble Learning**
- **CNN**.
- **RNN** implemented using a *LSTM*.

It can uses also a *random seed* to shuffle the data and use a different *training*, *validation* and *test* set respect to the ones used by ***ViraMiner***.

In [1]:
"""
Main libraries
"""

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import tensorflow as tf

import random
import gc

In [2]:
"""
For the class weight
"""
from sklearn.utils.class_weight import compute_class_weight

In [3]:
"""
For the ensamble learning
"""
from sklearn.svm import SVC

In [4]:
"""
Only for the metrics analysis
"""
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The **Onehot Encoding** that transform a *DNA sequence* to a vector of zeros and ones. It is useful expectially for the **CNN**.

In [6]:
"""
Functions for the onehot encoding
"""

def onehot_encoder(dataset):
    """
    Function that encodes a DNA dataset into a onehot encoding dataset.
    """
    onehot_dataset = [dna_onehot_encoder(dna_string) for dna_string in dataset]
    onehot_dataset_numpy = np.array(onehot_dataset)

    return onehot_dataset_numpy


def dna_onehot_encoder(dna_sequence):
    """
    Function that encodes a single DNA string into a onehot encoding string.
    """
    onehot_dict = {
        'A' : [1, 0, 0, 0],
        'C' : [0, 1, 0, 0],
        'G' : [0, 0, 1, 0],
        'T' : [0, 0, 0, 1]
    }
    encoder = [onehot_dict[nuc] for nuc in dna_sequence]

    return encoder

The **CNN** model: direct from the file *CNN.ipynb* avaiable in the GitHub repository.

In [7]:
"""
!- CNN Model
"""

class ConvNet(nn.Module):
    # We can use a differnet pool for each layer
    def __init__(self):
        super(ConvNet, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv1d(300, 200, kernel_size=2, padding=1),
            nn.BatchNorm1d(200),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out = nn.Dropout()

        self.layer2 = nn.Sequential(
            nn.Conv1d(200, 100, kernel_size=2, padding=1),
            nn.BatchNorm1d(100),
            nn.PReLU(),
            nn.AvgPool1d(2),
            nn.Dropout1d(0.45)
        )

        # I remove random connection to help the convergency
        self.drop_out = nn.Dropout()

        self.layer3 = nn.Sequential(
            nn.Conv1d(100, 75, kernel_size=2, padding=1),
            nn.BatchNorm1d(75),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        self.layer4 = nn.Sequential(
            nn.Conv1d(75, 50, kernel_size = 2, padding = 1),
            nn.BatchNorm1d(50),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout(0.3)
        )


        self.layer5 = nn.Sequential(
            nn.Conv1d(50, 32, kernel_size=2, padding=1),
            nn.BatchNorm1d(32),
            nn.PReLU(),
            nn.MaxPool1d(2),
            nn.Dropout1d(0.3)
        )

        self.linear1 = nn.Linear(32, 128)
        self.linear2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.layer1(x)
        # print("Layer 1: ", x.shape)
        x = self.layer2(x)
        # print("Layer 2: ", x.shape)
        x = self.layer3(x)
        # print("Layer 3: ", x.shape)
        x = self.layer4(x)
        # print("Layer 4: ", x.shape)
        x = self.layer5(x)
        # print("Layer 5: ", x.shape)
        # Flatten the output for the linear layer
        x = x.view(x.size(0), -1)
        x = self.linear1(x)
        x = self.linear2(x)
        # print("Linear: ", x.shape)
        return x

The **RNN** model: direct from the file *LSTM.ipynb* avaiable in the GitHub repository. We choose the *LSTM* over the *GRU* because it obtained an **higher** precision on the dataset and also is **less** computationally demanding.

In [8]:
class LSTM_Net (nn.Module):
    def __init__ (self):
        super(LSTM_Net, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=32, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)

        self.lstm = nn.LSTM(input_size=128, hidden_size=128, num_layers=2, batch_first=True, dropout=0.5)

        self.fc1 = nn.Linear(128, 256)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)

        x = x[:, -1, :]
        x = x.contiguous().view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

**MAIN**

Training phase

Extraction of the data from the *.cvc* files and **oversample** data from class 1.

In [9]:
"""
!- MAIN
"""

# Set the device to be used (GPU or CPU)
device = th.device("cuda" if th.cuda.is_available() else "cpu")
print("Device: ", device)

# Read the input from the cvc file
"""
If runs in local machine:

data_dir = os.path.abspath(os.path.join(os.getcwd(), 'data'))
train_data_path = os.path.join(data_dir, 'fullset_train.csv')
val_data_path = os.path.join(data_dir, 'fullset_validation.csv')
test_data_path = os.path.join(data_dir, 'fullset_test.csv')
"""

"""
If runs in Google Drive

rel_path_train = '/content/drive/MyDrive/Colab Notebooks/fullset_train.csv'
rel_path_val = '/content/drive/MyDrive/Colab Notebooks/fullset_validation.csv'
rel_path_test = '/content/drive/MyDrive/Colab Notebooks/fullset_test.csv'
"""

"""
If runs on Colab

rel_path_train = '/content/fullset_train.csv'
rel_path_val = '/content/fullset_validation.csv'
rel_path_test = '/content/fullset_test.csv'
"""

# Paste here your path to the datasets
rel_path_train = '/content/drive/MyDrive/Colab Notebooks/fullset_train.csv'
rel_path_val = '/content/drive/MyDrive/Colab Notebooks/fullset_validation.csv'
rel_path_test = '/content/drive/MyDrive/Colab Notebooks/fullset_test.csv'

# Training Set

# Read the input from the csv file
train_csv = pd.read_csv(rel_path_train, sep=",")
# Drop the NaN values
train_csv = train_csv.dropna()
# Describe the data
print(train_csv.describe())

# Get the data from the csv file
train_data = train_csv.values
# m = number of input samples
m = train_data.shape[0]

# Dataframe and upsample
data = {'sequence' : train_data[:m,1],
        'label' : train_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

count_class_0, count_class_1 = df['label'].value_counts()

df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]

# Oversample data from class 1 x10 -> ratio between class 1 and class 0 = 1/5
n_samples = count_class_0 / 5

print("Samples class 0: ", count_class_0)
print("Samples class 1: ", count_class_1)
print("Oversamples: ", int(n_samples))

df_class_1_oversampled = df_class_1.sample(int(n_samples), replace=True, random_state=42)
df_balanced = pd.concat([df_class_0, df_class_1_oversampled], axis=0)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract data
X_train = df_balanced['sequence'].values
Y_train = df_balanced['label'].values

# OneHot encoding for the training data
print("Start onehot encoding for the training data")
X_train = onehot_encoder(X_train)

# Convert the data to a tensor
X_train = th.from_numpy(X_train).to(device)
Y_train = th.tensor(Y_train).to(device)

print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

# Free memory
del train_csv, train_data, m, data, df, df_class_0, df_class_1, df_class_1_oversampled, df_balanced, n_samples
gc.collect()
th.cuda.empty_cache()

# Validation Set

# Read the input from the csv file
val_csv = pd.read_csv(rel_path_val, sep=",")
# Drop the NaN values
val_csv = val_csv.dropna()
# Describe the data
print(val_csv.describe())

val_data = val_csv.values
# m = number of input samples
m = val_data.shape[0]

# Dataframe and upsample
data = {'sequence' : val_data[:m,1],
        'label' : val_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

count_class_0, count_class_1 = df['label'].value_counts()

df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]

# Oversample data from class 1 x10 -> ratio between class 1 and class 0 = 1/5
n_samples = count_class_0 / 5

print("Samples class 0: ", count_class_0)
print("Samples class 1: ", count_class_1)
print("Oversamples: ", int(n_samples))

df_class_1_oversampled = df_class_1.sample(int(n_samples), replace=True, random_state=42)
df_balanced = pd.concat([df_class_0, df_class_1_oversampled], axis=0)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract data
X_val = df_balanced['sequence'].values
Y_val = df_balanced['label'].values

# OneHot encoding for the validation data
print("Start onehot encoding for the validation data")
X_val = onehot_encoder(X_val)

X_val = th.from_numpy(X_val).to(device)
Y_val = th.tensor(Y_val).to(device)

print("X_val shape", X_val.shape)
print("Y_val shape", Y_val.shape)

# Free memory
del val_csv, val_data, m, data, df, df_class_0, df_class_1, df_class_1_oversampled, df_balanced, n_samples
gc.collect()
th.cuda.empty_cache()

# Test

# Read the input from the csv file
test_csv = pd.read_csv(rel_path_test, sep=",")
# Drop the NaN values
test_csv = test_csv.dropna()
# Describe the data
print(test_csv.describe())

test_data = test_csv.values
# m = number of input samples
m = test_data.shape[0]

# Dataframe and upsample
data = {'sequence' : test_data[:m,1],
        'label' : test_data[:m,2].astype(np.int32) }

df = pd.DataFrame(data)

count_class_0, count_class_1 = df['label'].value_counts()

df_class_0 = df[df['label'] == 0]
df_class_1 = df[df['label'] == 1]

# Oversample data from class 1 x10 -> ratio between class 1 and class 0 = 1/5
n_samples = count_class_0 / 5

print("Samples class 0: ", count_class_0)
print("Samples class 1: ", count_class_1)
print("Oversamples: ", int(n_samples))

df_class_1_oversampled = df_class_1.sample(int(n_samples), replace=True, random_state=42)
df_balanced = pd.concat([df_class_0, df_class_1_oversampled], axis=0)
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Extract data
X_test = df_balanced['sequence'].values
Y_test = df_balanced['label'].values

# OneHot encoding for the test data
print("Start onehot encoding for the test data")
X_test = onehot_encoder(X_test)

X_test = th.from_numpy(X_test).to(device)
Y_test = th.tensor(Y_test).to(device)

print("X_test shape", X_test.shape)
print("Y_test shape", Y_test.shape)

# Free memory
del test_csv, test_data, m, data, df, df_class_0, df_class_1, df_class_1_oversampled, df_balanced, n_samples
gc.collect()
th.cuda.empty_cache()

Device:  cpu
                   0
count  211238.000000
mean        0.021142
std         0.143858
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Samples class 0:  206772
Samples class 1:  4466
Oversamples:  41354
Start onehot encoding for the training data
X_train shape:  torch.Size([248126, 300, 4])
Y_train shape:  torch.Size([248126])
                  0
count  26404.000000
mean       0.020224
std        0.140769
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Samples class 0:  25870
Samples class 1:  534
Oversamples:  5174
Start onehot encoding for the validation data
X_val shape torch.Size([31044, 300, 4])
Y_val shape torch.Size([31044])
                  0
count  26404.000000
mean       0.020868
std        0.142945
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max        1.000000
Samples class 0:  25853
Samples class 1:  551
Oversamples:  5

**Shuffle of the data** *(optional)* using a random seed and split data:
- **Train Set:** around the 80% of the total samples.
- **Validation Set:** around 10% of the total samples.
- **Test Set:** around 10% of the total samples.

In [10]:
"""
I merge the three tensor array to a big one and then, after a shuffle, I split the data into:
  - Training: 248126 data
  - Validation: 31034 data
  - Test: 31033 data

  Random seed: 42 (the most used)
"""

# Merge the arrays

print("Shape of X_train: ", X_train.shape)
print("Shape of X_val: ", X_val.shape)
print("Shape of X_test: ", X_test.shape)

X_data = th.concat((X_train, X_val), axis = 0).to(device)
X_data = th.concat((X_data, X_test), axis = 0).to(device)

print("Shape of data: ", X_data.shape)

# Free memory
del X_train, X_val, X_test
gc.collect()
th.cuda.empty_cache()

print("Shape of Y_train: ", Y_train.shape)
print("Shape of Y_val: ", Y_val.shape)
print("Shape of Y_test: ", Y_test.shape)

Y_data = th.concat((Y_train, Y_val), axis = 0).to(device)
Y_data = th.concat((Y_data, Y_test), axis = 0).to(device)

print("Shape of data: ", Y_data.shape)

# Class weight
classes_unique = np.unique(Y_data.cpu())
class_weights = compute_class_weight(class_weight='balanced', classes=classes_unique, y=[0,1])
class_weights = th.tensor(class_weights, dtype=th.float32)

# Free memory
del Y_train, Y_val, Y_test
gc.collect()
th.cuda.empty_cache()

# Random Seed
random_seed = 42
th.manual_seed(random_seed)

X_index_shuffle = th.randperm(X_data.size(0))
X_data_shuffled = X_data[X_index_shuffle]

Y_index_shuffle = th.randperm(Y_data.size(0))
Y_data_shuffled = Y_data[Y_index_shuffle]

print("X_data_shuffled shape: ", X_data_shuffled.shape)
print("Y_data_shuffled shape: ", Y_data_shuffled.shape)

# Split data into Training, Validation and Test
X_train = X_data_shuffled[:248126].cpu()
Y_train = Y_data_shuffled[:248126].cpu()
print("X_train shape: ", X_train.shape)
print("Y_train shape: ", Y_train.shape)

X_val = X_data_shuffled[211238:279160].cpu()
Y_val = Y_data_shuffled[211238:279160].cpu()
print("X_val shape: ", X_val.shape)
print("Y_val shape: ", Y_val.shape)

X_test = X_data_shuffled[279160:].cpu()
Y_test = Y_data_shuffled[279160:].cpu()
print("X_test shape: ", X_test.shape)
print("Y_test shape: ", Y_test.shape)

Shape of X_train:  torch.Size([248126, 300, 4])
Shape of X_val:  torch.Size([31044, 300, 4])
Shape of X_test:  torch.Size([31023, 300, 4])
Shape of data:  torch.Size([310193, 300, 4])
Shape of Y_train:  torch.Size([248126])
Shape of Y_val:  torch.Size([31044])
Shape of Y_test:  torch.Size([31023])
Shape of data:  torch.Size([310193])
X_data_shuffled shape:  torch.Size([310193, 300, 4])
Y_data_shuffled shape:  torch.Size([310193])
X_train shape:  torch.Size([248126, 300, 4])
Y_train shape:  torch.Size([248126])
X_val shape:  torch.Size([67922, 300, 4])
Y_val shape:  torch.Size([67922])
X_test shape:  torch.Size([31033, 300, 4])
Y_test shape:  torch.Size([31033])


In [11]:
"""
Class to create a Data Loader with X label and Y label together
"""
class CreateDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        label = self.labels[idx]
        return data_point, label

# Create the Dataset
train_dataset = CreateDataset(X_train, Y_train)
val_dataset = CreateDataset(X_val, Y_val)
test_dataset = CreateDataset(X_test, Y_test)

# Batch size
batch_dim = 5000

# Create the Data Loader
train_loader = DataLoader(train_dataset, batch_size=batch_dim, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_dim, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_dim, shuffle=True)

# Free memory
del X_train, X_val, Y_val, X_test
gc.collect()
th.cuda.empty_cache()

*Train* of the **CNN** using the *Validation* set to check the performance.

In [ ]:
print("Start training the CNN")

model_CNN = ConvNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss(weight=class_weights)
optimizer = th.optim.AdamW(model_CNN.parameters(), lr=0.001)
scheduler = th.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True)

# Early stopping parameters
patience = 5  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches

for epoch in range(num_epochs):
    model_CNN.train()
    running_loss = 0.0
    i = 0
    for X_batch, Y_batch in train_loader:
        X_batch = X_batch.float()
        Y_batch = Y_batch.long()

        # Forward pass
        outputs = model_CNN(X_batch.to(device))
        loss = criterion(outputs, Y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        i += 1

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

    # Validation the model
    model_CNN.eval()
    val_loss = 0.0
    j = 0
    with th.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.float()
            Y_batch = Y_batch.long()

            outputs = model_CNN(X_batch.to(device))
            loss = criterion(outputs, Y_batch.to(device))
            scheduler.step(val_loss)

            val_loss += loss.item()
            j += 1

            # Free memory
            del X_batch, Y_batch
            th.cuda.empty_cache()

    # Losses
    running_loss = running_loss/i
    val_loss = val_loss/j

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the CNN")

# Free memory
del i, j, running_loss, val_loss
gc.collect()
th.cuda.empty_cache()

Start training the CNN


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch [1/100], Train Loss: 0.4798, Val Loss: 0.4552


**(OPTIONAL):** *Test* results for the **CNN**.

In [ ]:
print("Start testing the CNN")

# Model ready for the evaluation
model_CNN.eval()

# Arrays to save the results
all_preds = []
all_labels = []

# Testing the model
with th.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        outputs = model_CNN(X_batch)
        _, predicted = th.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(Y_batch.cpu().numpy())

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

# Convert the tensor to use scikit learn metrics
y_true_CNN = all_labels
y_pred_CNN = all_preds

# Metrics
accuracy_CNN = accuracy_score(y_true_CNN, y_pred_CNN)
precision_CNN = precision_score(y_true_CNN, y_pred_CNN, average='weighted', zero_division=0)
recall_CNN = recall_score(y_true_CNN, y_pred_CNN, average='weighted')
f1_CNN = f1_score(y_true_CNN, y_pred_CNN, average='weighted')

print(f'Accuracy: {accuracy_CNN:.6f}')
print(f'Precision: {precision_CNN:.6f}')
print(f'Recall: {recall_CNN:.6f}')
print(f'F1-score: {f1_CNN:.6f}')

print("End testing the CNN")

# Free memory
del all_labels, all_preds, y_true_CNN, y_pred_CNN, accuracy_CNN, precision_CNN, recall_CNN, f1_CNN
gc.collect()
th.cuda.empty_cache()

*Train* of the **RNN** using the *Validation* set to check the performance.

In [ ]:
print("Start training the LSTM")

# Model, loss function and optimizer
model_LSTM = LSTM_Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = th.optim.Adam(model_LSTM.parameters(), lr=0.001)

# Early stopping parameters
patience = 5  # CHECK THE VALUE!!
best_val_loss = float('inf')
counter = 0

# Training the model
num_epochs = 100  # Hope to reach convergence before 100 epoches

for epoch in range(num_epochs):
    model_LSTM.train()
    running_loss = 0.0
    i = 0
    for X_batch, Y_batch in train_loader:

        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        # Forward pass
        outputs = model_LSTM(X_batch.to(device))
        loss = criterion(outputs, Y_batch.to(device))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        i += 1

        # Free memory
        del X_batch, Y_batch
        th.cuda.empty_cache()

    # Validation
    model_LSTM.eval()
    val_loss = 0.0
    j = 0
    with th.no_grad():
        for X_batch, Y_batch in val_loader:
            X_batch = X_batch.to(device).float()
            Y_batch = Y_batch.to(device).long()

            outputs = model_LSTM(X_batch)
            loss = criterion(outputs, Y_batch)

            val_loss += loss.item()
            j += 1

            # Free memory
            del X_batch, Y_batch
            th.cuda.empty_cache()

    # Losses
    running_loss = running_loss/i
    val_loss = val_loss/j

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Check for early stopping
    if val_loss <= best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

print("End training the LSTM")

# Free memory
del i, j, running_loss, val_loss
gc.collect()
th.cuda.empty_cache()

**(OPTIONAL):** *Test* results forthe **RNN**.

In [ ]:
print("Start testing the LSTM")

# Model ready for the evaluation
model_LSTM.eval()

# Arrays to save the results
all_preds = []
all_labels = []

# Testing the model
with th.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device).float()
        Y_batch = Y_batch.to(device).long()

        outputs = model_CNN(X_batch)
        _, predicted = th.max(outputs, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(Y_batch.cpu().numpy())

# Convert the tensor to use scikit learn metrics
y_true_LSTM = all_labels
y_pred_LSTM = all_preds

# Metrics
accuracy_LSTM = accuracy_score(y_true_LSTM, y_pred_LSTM)
precision_LSTM = precision_score(y_true_LSTM, y_pred_LSTM, average='weighted', zero_division=0)
recall_LSTM = recall_score(y_true_LSTM, y_pred_LSTM, average='weighted')
f1_LSTM = f1_score(y_true_LSTM, y_pred_LSTM, average='weighted')

print(f'Accuracy: {accuracy_LSTM:.6f}')
print(f'Precision: {precision_LSTM:.6f}')
print(f'Recall: {recall_LSTM:.6f}')
print(f'F1-score: {f1_LSTM:.6f}')

print("End testing the LSTM")

# Free memory
del all_labels, all_preds, y_true_LSTM, y_pred_LSTM, accuracy_LSTM, precision_LSTM, recall_LSTM, f1_LSTM
gc.collect()
th.cuda.empty_cache()

Implementation of the **Ensemble Learning** using an **SVM** (stacking ensemble learing).

In [ ]:
"""
An SVM takes in input the results from the CNN and RNN and then it preforms the final classification.

I have to use skitlearn to do the ensable learing because the library is only avaiable on it!
"""

"""
Function that, given a model and a data loader, it returns all the predictions (outputs)
"""
def get_predictions(model_pred, data_loader):
    model_pred.eval()
    all_preds = []
    with th.no_grad():
        for inputs, _ in data_loader:
            outputs = model_pred(inputs.float().to(device))
            all_preds.append(outputs.cpu().numpy())
    return np.concatenate(all_preds)

# Prepare the prediction for the training model
CNN_train_preds = get_predictions(model_CNN, train_loader)
LSTM_train_preds = get_predictions(model_LSTM, train_loader)

# Prepare the prediction for the testing model
CNN_test_preds = get_predictions(model_CNN, test_loader)
LSTM_test_preds = get_predictions(model_LSTM, test_loader)

# Concatenate the two inputs
X_train_meta = np.hstack((CNN_train_preds, LSTM_train_preds))
X_test_meta = np.hstack((CNN_test_preds, LSTM_test_preds))

print("Shape of X_train_meta: ", X_train_meta.shape)
print("Shape of Y_train: ", Y_train.shape)

print("Shape of X_test_meta: ", X_test_meta.shape)
print("Shape of Y_test: ", Y_test.shape)

# Free memory
gc.collect()
th.cuda.empty_cache()

In [ ]:
# We use a polynomial SVM for the final classification
model_SVM = SVC(kernel = 'poly', probability = True)

train_dataset = CreateDataset(X_train_meta, Y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_dim, shuffle=True)

for X_batch, Y_batch in train_loader:
    model_SVM.fit(X_batch, Y_batch.ravel())

**Test** of the performance.

In [ ]:
print("Start testing the final model")

# I convert the data (if needed)
y_true = Y_test.cpu().numpy()

# Testing the meta-model: SVM
y_pred = model_SVM.predict(X_test_meta)
y_pred_prob = model_SVM.predict_proba(X_test_meta)[:, 1]

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')
auroc = roc_auc_score(y_true, y_pred_prob)

print(f'Accuracy: {accuracy:.6f}')
print(f'Precision: {precision:.6f}')
print(f'Recall: {recall:.6f}')
print(f'F1-score: {f1:.6f}')
print(f'AUROC: {auroc:.6f}')

print("End testing the final model")